In [93]:
import findspark

findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import year, col, concat, lit

In [94]:
spark = SparkSession.builder.master("local[*]").appName("join").getOrCreate()

In [95]:
drivers_df = spark.read.csv("dataset/drivers.csv", header=True, inferSchema=True)
standings_df = spark.read.csv("dataset/driver_standings.csv", header=True, inferSchema=True)
races_df = spark.read.csv("dataset/races.csv", header=True, inferSchema=True)

In [96]:
joined_df = (
    drivers_df[["driverId", "dob", "forename", "surname", "nationality"]]
    .withColumn("dob", year("dob"))
    .withColumn("name", concat(col("forename"), lit(" "), col("surname")))
    .drop("forename")
    .drop("surname")
    .join(standings_df[["raceId", "driverId"]], on=["driverId"])
    .join(races_df[["raceId", "year"]], on=["raceId"])
    .filter(races_df["year"] <= 2020)
    .rdd.map(lambda x: (x["driverId"], x))
    .sortByKey()
    .map(lambda kv: (kv[0], (kv[1]["year"], kv[1]["dob"])))
    .reduceByKey(lambda x, y: max(x, y))
    .map(lambda kv: (kv[0], kv[1][0] - kv[1][1]))
    .filter(lambda x: x[1] >= 29)
    .map(lambda x: (x[1], 1))
    .reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
)
joined_df[0] / joined_df[1]

36.00481540930979